In [1]:
# Import objects from pyomo package
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           Constraint,
                           Expression,
                           TransformationFactory,
                           value,
                           units as pyunits)

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import idaes logger to set output levels
import idaes.logger as idaeslog

from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

# Todo: import Flash unit model from idaes.generic_models.unit_models
from idaes.generic_models.unit_models import Flash

In [2]:
from idaes.core.util.model_statistics import degrees_of_freedom, large_residuals_set

import pyomo.contrib.parmest.parmest as parmest

In [3]:
# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

# Import Pandas
import pandas as pd

In [4]:
from HFCs_bmimPF6_PR import configuration

# Load data from csv
data = pd.read_csv('HFCSIL_hetereo_298.csv')

In [5]:
def PR_model(data):
    
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})

    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(298.15)
    m.fs.state_block.pressure.fix(500000)
    m.fs.state_block.mole_frac_comp["R32"].fix(0.25)
    m.fs.state_block.mole_frac_comp["R125"].fix(0.05)
    m.fs.state_block.mole_frac_comp["bmimPF6"].fix(0.7)
    
    m.fs.properties.PR_kappa_A['R32', 'R125'].fix(-0.7)
    m.fs.properties.PR_kappa_A['R125', 'R32'].fix(-0.002535)
    m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].fix(-0.03)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].fix(-0.15)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].fix(0.4)
    m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].fix(-0.002)
    
    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix at actual temperature
    m.fs.state_block.temperature.fix(float(data["temperature"]))
#     m.fs.state_block.pressure.fix(float(data["pressure"]))
    m.fs.state_block.pressure.unfix()
    m.fs.state_block.mole_frac_comp["R125"].unfix()
    m.fs.state_block.mole_frac_comp["R32"].unfix()
    m.fs.state_block.mole_frac_comp["bmimPF6"].unfix()
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R32'].fix(float(data["y_R32"]))
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R125'].fix(float(data["y_R125"]))   
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R32'].fix(float(data["x_R32"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R125'].fix(float(data["x_R125"])) 
    m.fs.state_block.mole_frac_phase_comp['Liq', 'bmimPF6'].fix(float(data["x_IL"])) 
    
    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa_A['R32', 'R125'].setlb(-3)
    m.fs.properties.PR_kappa_A['R32', 'R125'].setub(3)

    m.fs.properties.PR_kappa_A['R125', 'R32'].setlb(-3)
    m.fs.properties.PR_kappa_A['R125', 'R32'].setub(3)

    m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].setlb(-3)
    m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].setub(3)

    m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].setlb(-3)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].setub(3)
    
    m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].setlb(-3)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].setub(3)

    m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].setlb(-3)
    m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].setub(3)
    
    # Return initialized flash model
    return m

In [6]:
import pytest

test_data = {"temperature": 298.15, "pressure": 100000, "x_R32":0.7312, "x_IL":0.7312,  
             "x_R125":0.2688}

m = PR_model(test_data)

DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

2021-09-17 16:59:04 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
The initial DOF is 1


In [7]:
variable_name = [
                 "fs.properties.PR_kappa_A['R32', 'R125']",
                 "fs.properties.PR_kappa_A['R125', 'R32']",
                 "fs.properties.PR_kappa_A['R32', 'bmimPF6']",
                 "fs.properties.PR_kappa_A['bmimPF6', 'R32']",
                 "fs.properties.PR_kappa_A['bmimPF6', 'R125']",
                 "fs.properties.PR_kappa_A['R125', 'bmimPF6']"]

In [8]:
def SSE(m, data):
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)
    return expr*1e-7

# def SSE(m, data):
#     expr = (
#             (float(data["y_R125"]) -
#              m.fs.state_block.mole_frac_phase_comp["Vap", "R125"])**2
# #             + (float(data["x_R125"]) -
# #              m.fs.state_block.mole_frac_phase_comp["Liq", "R125"])**2
#            )

In [9]:
solver_options = {'tol': 1e-7}
pest = parmest.Estimator(PR_model, data, variable_name, SSE, solver_options)
# solver_options = {'tol': 1e-6, 'max_iter':100}

obj_value, parameters = pest.theta_est()

2021-09-17 16:59:09 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-09-17 16:59:13 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-09-17 16:59:18 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-09-17 16:59:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-09-17 16:59:26 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-09-17 16:59:30 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-09-17 16:59:34 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-09-17 16:59:38 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-09-17 16:59:42 [INFO] idaes

  61  1.8898849e+04 5.12e-01 6.95e+03  -1.0 3.98e+05    -  3.94e-02 1.47e-02h  1
  62  1.8893877e+04 5.09e-01 8.49e+03  -1.0 3.90e+05    -  5.56e-02 3.23e-03f  1
  63  1.8844181e+04 4.84e+01 1.48e+04  -1.0 3.87e+05    -  1.76e-03 2.04e-01f  2
  64  1.8835835e+04 4.40e+01 1.51e+04  -1.0 2.73e+05    -  4.55e-02 1.16e-01f  1
  65  1.8875976e+04 3.84e+01 1.64e+04  -1.0 2.38e+05    -  2.00e-01 2.15e-01h  2
  66  1.8878135e+04 3.77e+01 1.61e+04  -1.0 1.82e+05    -  1.97e-01 1.91e-02h  5
  67  1.8879731e+04 3.63e+01 1.55e+04  -1.0 1.79e+05    -  5.02e-01 3.72e-02h  4
  68r 1.8879731e+04 3.63e+01 1.00e+03   1.6 0.00e+00   0.3 0.00e+00 3.63e-07R  5
  69r 1.8879730e+04 1.38e+01 3.66e+03   1.6 2.84e+04    -  1.16e-03 1.07e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  70  1.8879848e+04 1.38e+01 5.11e+03  -1.0 1.67e+05    -  3.06e-01 1.67e-04h  1
  71  1.8880696e+04 1.38e+01 4.75e+03  -1.0 1.56e+05    -  3.62e-01 1.04e-03h  9
  72  1.8880713e+04 1.38e+01

 152  1.0661359e+04 5.08e+00 3.59e+15  -1.0 8.11e+02   7.0 4.92e-01 3.33e-02h  5
 153  1.0661358e+04 5.07e+00 3.58e+15  -1.0 6.70e+02   9.3 5.29e-02 1.43e-03h  6
 154  1.0661300e+04 2.75e+00 2.87e+15  -1.0 3.00e+02   8.8 3.17e-01 4.84e-01h  1
 155  1.0661299e+04 2.72e+00 2.84e+15  -1.0 1.51e+02  10.1 9.90e-01 9.39e-03h  1
 156  1.0661299e+04 2.72e+00 2.84e+15  -1.0 1.46e+02  10.5 3.05e-01 9.48e-05h  1
 157r 1.0661299e+04 2.72e+00 1.00e+03   0.4 0.00e+00  11.0 0.00e+00 4.74e-07R  2
 158r 1.0661301e+04 1.64e+00 1.34e+03   0.4 2.72e+03    -  1.71e-03 9.91e-04f  1
 159  1.0586568e+04 7.46e-01 1.21e+04  -1.0 5.53e+03    -  8.79e-01 9.94e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 160  1.0586568e+04 7.45e-01 2.82e+11  -1.0 1.54e+03  10.5 2.27e-02 1.71e-03h  4
 161  1.0586576e+04 1.40e+01 3.26e+15  -1.0 2.62e+03  10.0 1.16e-02 1.02e-02f  2
 162  1.0586576e+04 1.36e+01 3.17e+15  -1.0 9.84e+01   9.5 4.88e-01 2.94e-02h  5
 163  1.0586553e+04 2.80e-01

 245  3.7634625e+03 1.25e+01 9.11e+00  -5.7 6.94e+04  -7.5 1.42e-01 4.50e-01f  1
 246  3.6288760e+03 1.58e+01 2.13e+00  -5.7 2.70e+04  -7.1 1.00e+00 2.25e-01f  1
 247  3.4022223e+03 1.19e+01 2.19e+00  -5.7 1.05e+04  -6.6 6.00e-02 1.00e+00f  1
 248  3.3570039e+03 7.94e+00 1.76e+00  -5.7 3.88e+03  -6.2 1.60e-01 5.49e-01f  1
 249  3.3262839e+03 4.88e+00 1.37e+00  -5.7 1.46e+03  -5.8 4.71e-05 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 250  3.3261042e+03 2.25e+00 5.42e-01  -5.7 8.53e+00  -3.6 3.85e-01 1.00e+00h  1
 251  3.3258740e+03 5.78e+00 1.08e+01  -5.7 2.56e+01  -4.0 1.96e-01 4.28e-01h  1
 252  3.3242589e+03 3.70e+00 9.13e+00  -5.7 7.67e+01  -4.5 1.00e+00 1.00e+00h  1
 253  3.3242277e+03 4.83e+00 2.76e+00  -5.7 3.85e+00  -3.2 2.29e-01 4.14e-01h  1
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
 254  3.3242273e+03 4.40e+00 1.24e+01  -5.7 8.49e-01  -1.9 5.42e-05 9.83e-02h  2
 255  3.3242238e+03 6.10e+00 7

 317  2.6548446e+03 2.25e+00 4.02e+02  -5.7 2.83e+05    -  6.03e-02 8.93e-03f  1
 318  2.6138429e+03 2.23e+00 3.64e+02  -5.7 2.80e+05    -  9.46e-02 7.81e-03f  1
 319  2.5270834e+03 2.85e+01 3.55e+02  -5.7 2.78e+05    -  2.42e-02 1.69e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 320  2.0493465e+03 2.57e+01 1.81e+02  -5.7 2.73e+05    -  4.92e-01 1.00e-01f  1
 321  2.0493302e+03 2.57e+01 1.76e+02  -5.7 2.46e+05    -  2.32e-02 4.00e-06f  1
 322  2.0274827e+03 2.49e+02 1.76e+02  -5.7 2.46e+05    -  2.72e-03 5.37e-03f  1
 323  1.6012205e+03 2.21e+02 5.52e+05  -5.7 2.45e+05    -  3.48e-07 1.12e-01f  1
 324  1.6012365e+03 2.16e+02 5.41e+05  -5.7 2.51e+03  -1.9 9.59e-02 2.25e-02h  1
 325  1.6014472e+03 1.51e+02 5.09e+05  -5.7 2.46e+03  -2.4 2.70e-01 3.02e-01h  1
 326  1.6015201e+03 1.27e+02 4.77e+05  -5.7 1.71e+03  -2.8 1.24e-01 1.60e-01h  1
 327r 1.6015201e+03 1.27e+02 9.99e+02   2.1 0.00e+00  -3.3 0.00e+00 4.84e-07R  3
 328r 1.6015202e+03 1.25e+02

 406  3.3312596e+01 4.23e+00 2.67e+04  -5.7 1.00e+04    -  5.21e-01 8.76e-05h  1
 407  3.0928809e+01 1.35e+01 8.06e+04  -5.7 6.05e+04    -  9.85e-02 1.53e-01H  1
 408  3.1560236e+01 1.32e+01 7.85e+04  -5.7 4.48e+04    -  2.48e-02 7.30e-02H  1
 409  3.2814368e+01 1.28e+01 7.55e+04  -5.7 4.17e+04    -  2.00e-01 1.33e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 410  2.8702547e+01 1.37e+01 9.44e+04  -5.7 7.59e+03    -  7.47e-02 1.53e-01H  1
 411  2.5728255e+01 1.29e+01 5.74e+08  -5.7 6.13e+04    -  5.73e-06 2.04e-01f  1
 412  2.5729310e+01 1.29e+01 5.74e+08  -5.7 4.43e+03   0.7 6.79e-06 8.50e-04h  4
 413  2.5776998e+01 1.27e+01 5.64e+08  -5.7 6.11e+03   0.3 2.89e-05 2.10e-02h  1
 414  2.6571156e+01 1.07e+01 5.13e+08  -5.7 5.99e+03  -0.2 1.47e-03 3.42e-01h  1
 415  2.6578419e+01 1.06e+01 5.11e+08  -5.7 3.96e+03  -0.7 1.92e-04 4.60e-03h  1
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
 416  2.6803546e+01 9.52e+00 4

 481  1.9557261e+01 3.30e-01 9.91e+11  -5.7 3.18e+04  -3.9 2.78e-01 1.57e-01w  1
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
 482  1.8733364e+01 1.12e-01 1.49e+12  -5.7 3.68e+04  -4.4 2.78e-01 3.93e-02h  2
 483  1.8933468e+01 1.13e-01 3.89e+12  -5.7 3.28e+04  -4.0 9.50e-01 3.08e-02h  3
 484  1.9296016e+01 1.19e-01 3.07e+12  -5.7 3.39e+04  -4.5 3.26e-01 4.93e-02h  2
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
 485  1.9631118e+01 1.22e-01 2.68e+12  -5.7 3.68e+04  -4.9 1.52e-01 3.90e-02h  4
 486  1.9892911e+01 1.23e-01 1.77e+12  -5.7 3.60e+04  -4.5 9.20e-01 2.81e-02h  2
 487  2.0286275e+01 1.25e-01 3.74e+12  -5.7 3.64e+04  -4.1 7.78e-01 3.88e-02h  1
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
 488  2.0546021e+01 1.24e-01 3.70e+12  -5.7 3.78e+04  -4.6 3.74e-04 2.34e-02h  5
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
 489  2.0776477e+01 1.24e-01 3.56e+1

 536  2.7082304e+01 2.40e-02 3.14e+10  -5.7 4.84e+04    -  5.90e-01 1.93e-02h  5
 537  2.7287587e+01 2.38e-02 1.95e+10  -5.7 4.84e+04    -  4.14e-01 9.50e-03h  6
 538  2.7499569e+01 2.35e-02 1.20e+10  -5.7 4.83e+04    -  4.44e-01 9.51e-03h  6
 539  2.7719454e+01 2.33e-02 8.11e+09  -5.7 4.81e+04    -  4.23e-01 9.57e-03h  6
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 540  2.7948067e+01 2.31e-02 5.61e+09  -5.7 4.81e+04    -  4.70e-01 9.64e-03h  6
 541  2.8186109e+01 2.29e-02 4.58e+09  -5.7 4.81e+04    -  3.68e-01 9.73e-03h  6
 542  3.8596981e+01 2.63e-01 2.59e+09  -5.7 4.81e+04    -  6.51e-01 3.15e-01w  1
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
 543  2.8434224e+01 2.27e-02 3.49e+09  -5.7 5.49e+04    -  6.51e-01 9.83e-03h  5
 544  2.8693981e+01 2.25e-02 3.32e+09  -5.7 4.82e+04    -  2.94e-01 9.97e-03h  6
 545  2.8966998e+01 2.22e-02 3.04e+09  -5.7 4.82e+04    -  9.56e-01 1.01e-02h  6
 546  2.9262332e+01 2.20e-02 3

 601  3.0397591e+01 5.88e-05 2.55e+03  -5.7 3.55e+03    -  8.91e-01 6.54e-04f  1
 602r 3.0397591e+01 5.88e-05 1.00e+03  -4.2 0.00e+00    -  0.00e+00 1.74e-08R  2
 603r 3.0397592e+01 1.34e-05 9.93e+02  -4.2 9.73e-01    -  2.55e-01 9.90e-04f  1
 604  3.0397570e+01 1.34e-05 6.07e+04  -5.7 3.55e+03    -  8.91e-01 2.74e-05h  1
 605r 3.0397570e+01 1.34e-05 1.00e+03  -4.9 0.00e+00    -  0.00e+00 3.73e-07R  8
 606r 3.0397570e+01 3.39e-06 9.94e+02  -4.9 1.44e+00    -  9.84e-01 9.90e-04f  1
 607  3.0397568e+01 3.39e-06 2.66e+05  -5.7 3.55e+03    -  8.91e-01 3.14e-06h  2
 608r 3.0397568e+01 3.39e-06 1.00e+03  -5.5 0.00e+00    -  0.00e+00 3.76e-07R  6
 609r 3.0397568e+01 2.54e-06 9.98e+02  -5.5 2.07e+00    -  9.84e-01 9.90e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 610  3.0397567e+01 2.54e-06 3.53e+05  -5.7 3.55e+03    -  8.91e-01 2.36e-06h  2
 611  3.0109697e+01 5.71e-03 1.51e+05  -5.7 1.15e+04    -  5.75e-01 7.55e-02f  1
 612r 3.0109697e+01 5.71e-03

In [10]:
print("The SSE at the optimal solution is %0.6f" % obj_value)
print()
print("The values for the parameters are as follows:")
for k,v in parameters.items():
    print(k, "=", v)

The SSE at the optimal solution is 30.188277

The values for the parameters are as follows:
fs.properties.PR_kappa_A[R32,R125] = -2.2038402366210463
fs.properties.PR_kappa_A[R125,R32] = -0.37422827716994905
fs.properties.PR_kappa_A[R32,bmimPF6] = -0.07050227904941613
fs.properties.PR_kappa_A[bmimPF6,R32] = -0.2809715341057735
fs.properties.PR_kappa_A[bmimPF6,R125] = 2.1132383499378498
fs.properties.PR_kappa_A[R125,bmimPF6] = -0.05892020328285142


In [11]:
print(large_residuals_set(pest.ef_instance))

ComponentSet(['Scenario24.fs.state_block.eq_temperature_bubble[Vap,Liq,R125] (id=1759670936768)'])
